In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import glob
import os
from sklearn.model_selection import cross_val_score
import abc
import re
import nltk
from sklearn import model_selection 
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
import datetime
import pylab as plt
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.feature_selection import VarianceThreshold
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
def createComparableDataset(_df):
    _df.date = pd.to_datetime(_df.date)
    _df = _df[_df['date'] >= pd.to_datetime('2015/05/04')]
    _df = _df[_df['date'] <= pd.to_datetime('2019/03/28')]
    return _df

In [4]:
def createTrainingAndTestingSet(_df):
    train = _df[_df['date'] < pd.to_datetime('2019/01/01')]
    test = _df[_df['date'] >= pd.to_datetime('2019/01/01')]
    return train ,test

In [5]:
def getStartEndDateShape(dfz):
    startDate = str(min(dfz['date']))
    endDate = str(max(dfz['date']))
    print('startDate', startDate)
    print('endDate', endDate)
    print('shape', dfz.shape)
    print('---------------------------')

In [6]:
def getFeaturesAndTargetRemoveDate(_df):
    try:
        train_y = _df['target']
        train = _df.drop(['date','target'],axis =1)
    except:
        train_y = _df['Target']
        train = _df.drop(['date','Target'],axis =1)        
    return train, train_y

In [7]:
trainingData = pd.read_csv('train_SEFnNSB_bert_tnfi.csv')
testingData =  pd.read_csv('test_SEFnNSB_bert_tnfi.csv')

In [8]:
selectedFeatureList =['date', 'target','Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'sma2',
 'sma2_1',
 'sma2_increment',
 'sma2_1_increment',
 'vol_increment',
 'vol_rel_increment',
 'open_1',
 'open_incr',
 'ma7',
 'ma21',
 '26ema',
 '12ema',
 'MACD',
 '30 Day MA',
 '30 Day STD',
 'upper_band',
 'lower_band',
 'ema',
 'momentum',
 'log_momentum',
 'fft',
 'absolute',
 'angle',
 'news_len',
 'lm_positive_x',
 'lm_negative_x',
 'lm_polarity_x',
 'lm_subjectivity_x',
 'actvie_x',
 'passive_x',
 'weak_x',
 'strong_x',
 'anger_x',
 'joy_x',
 'suprise_x',
 'sadness_x',
 'disgust_x',
 'anticipation_x',
 'fear_x',
 'trust_x',
 'tb_polarity_x',
 'tb_subjectivity_x',
 'hiv4_positive_x',
 'hiv4_negative_x',
 'hiv4_polarity_x',
 'hiv4_subjectivity_x',
  '0_y', '1_y', '2_y', '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y', '12_y', '13_y', '14_y', '15_y', '16_y', '17_y', '18_y', '19_y', '20_y', '21_y', '22_y', '23_y', '24_y', '25_y', '26_y', '27_y', '28_y', '29_y', '30_y', '31_y', '32_y', '33_y', '34_y', '35_y', '36_y', '37_y', '38_y', '39_y', '40_y', '41_y', '42_y', '43_y', '44_y', '45_y', '46_y', '47_y', '48_y', '49_y', '50_y', '51_y', '52_y', '53_y', '54_y', '55_y', '56_y', '57_y', '58_y', '59_y', '60_y', '61_y', '62_y', '63_y', '64_y', '65_y', '66_y', '67_y', '68_y', '69_y', '70_y', '71_y', '72_y', '73_y', '74_y', '75_y', '76_y', '77_y', '78_y', '79_y', '80_y', '81_y', '82_y', '83_y', '84_y', '85_y', '86_y', '87_y', '88_y', '89_y', '90_y', '91_y', '92_y', '93_y', '94_y', '95_y', '96_y', '97_y', '98_y', '99_y', '100_y', '101_y', '102_y', '103_y', '104_y', '105_y', '106_y', '107_y', '108_y', '109_y', '110_y', '111_y', '112_y', '113_y', '114_y', '115_y', '116_y', '117_y', '118_y', '119_y', '120_y', '121_y', '122_y', '123_y', '124_y', '125_y', '126_y', '127_y', '128_y', '129_y', '130_y', '131_y', '132_y', '133_y', '134_y', '135_y', '136_y', '137_y', '138_y', '139_y', '140_y', '141_y', '142_y', '143_y', '144_y', '145_y', '146_y', '147_y', '148_y', '149_y', '150_y', '151_y', '152_y', '153_y', '154_y', '155_y', '156_y', '157_y', '158_y', '159_y', '160_y', '161_y', '162_y', '163_y', '164_y', '165_y', '166_y', '167_y', '168_y', '169_y', '170_y', '171_y', '172_y', '173_y', '174_y', '175_y', '176_y', '177_y', '178_y', '179_y', '180_y', '181_y', '182_y', '183_y', '184_y', '185_y', '186_y', '187_y', '188_y', '189_y', '190_y', '191_y', '192_y', '193_y', '194_y', '195_y', '196_y', '197_y', '198_y', '199_y', '200_y', '201_y', '202_y', '203_y', '204_y', '205_y', '206_y', '207_y', '208_y', '209_y', '210_y', '211_y', '212_y', '213_y', '214_y', '215_y', '216_y', '217_y', '218_y', '219_y', '220_y', '221_y', '222_y', '223_y', '224_y', '225_y', '226_y', '227_y', '228_y', '229_y', '230_y', '231_y', '232_y', '233_y', '234_y', '235_y', '236_y', '237_y', '238_y', '239_y', '240_y', '241_y', '242_y', '243_y', '244_y', '245_y', '246_y', '247_y', '248_y', '249_y', '250_y', '251_y', '252_y', '253_y', '254_y', '255_y', '256_y', '257_y', '258_y', '259_y', '260_y', '261_y', '262_y', '263_y', '264_y', '265_y', '266_y', '267_y', '268_y', '269_y', '270_y', '271_y', '272_y', '273_y', '274_y', '275_y', '276_y', '277_y', '278_y', '279_y', '280_y', '281_y', '282_y', '283_y', '284_y', '285_y', '286_y', '287_y', '288_y', '289_y', '290_y', '291_y', '292_y', '293_y', '294_y', '295_y', '296_y', '297_y', '298_y', '299_y', '300_y', '301_y', '302_y', '303_y', '304_y', '305_y', '306_y', '307_y', '308_y', '309_y', '310_y', '311_y', '312_y', '313_y', '314_y', '315_y', '316_y', '317_y', '318_y', '319_y', '320_y', '321_y', '322_y', '323_y', '324_y', '325_y', '326_y', '327_y', '328_y', '329_y', '330_y', '331_y', '332_y', '333_y', '334_y', '335_y', '336_y', '337_y', '338_y', '339_y', '340_y', '341_y', '342_y', '343_y', '344_y', '345_y', '346_y', '347_y', '348_y', '349_y', '350_y', '351_y', '352_y', '353_y', '354_y', '355_y', '356_y', '357_y', '358_y', '359_y', '360_y', '361_y', '362_y', '363_y', '364_y', '365_y', '366_y', '367_y', '368_y', '369_y', '370_y', '371_y', '372_y', '373_y', '374_y', '375_y', '376_y', '377_y', '378_y', '379_y', '380_y', '381_y', '382_y', '383_y', '384_y', '385_y', '386_y', '387_y', '388_y', '389_y', '390_y', '391_y', '392_y', '393_y', '394_y', '395_y', '396_y', '397_y', '398_y', '399_y', '400_y', '401_y', '402_y', '403_y', '404_y', '405_y', '406_y', '407_y', '408_y', '409_y', '410_y', '411_y', '412_y', '413_y', '414_y', '415_y', '416_y', '417_y', '418_y', '419_y', '420_y', '421_y', '422_y', '423_y', '424_y', '425_y', '426_y', '427_y', '428_y', '429_y', '430_y', '431_y', '432_y', '433_y', '434_y', '435_y', '436_y', '437_y', '438_y', '439_y', '440_y', '441_y', '442_y', '443_y', '444_y', '445_y', '446_y', '447_y', '448_y', '449_y', '450_y', '451_y', '452_y', '453_y', '454_y', '455_y', '456_y', '457_y', '458_y', '459_y', '460_y', '461_y', '462_y', '463_y', '464_y', '465_y', '466_y', '467_y', '468_y', '469_y', '470_y', '471_y', '472_y', '473_y', '474_y', '475_y', '476_y', '477_y', '478_y', '479_y', '480_y', '481_y', '482_y', '483_y', '484_y', '485_y', '486_y', '487_y', '488_y', '489_y', '490_y', '491_y', '492_y', '493_y', '494_y', '495_y', '496_y', '497_y', '498_y', '499_y', '500_y', '501_y', '502_y', '503_y', '504_y', '505_y', '506_y', '507_y', '508_y', '509_y', '510_y', '511_y', '512_y', '513_y', '514_y', '515_y', '516_y', '517_y', '518_y', '519_y', '520_y', '521_y', '522_y', '523_y', '524_y', '525_y', '526_y', '527_y', '528_y', '529_y', '530_y', '531_y', '532_y', '533_y', '534_y', '535_y', '536_y', '537_y', '538_y', '539_y', '540_y', '541_y', '542_y', '543_y', '544_y', '545_y', '546_y', '547_y', '548_y', '549_y', '550_y', '551_y', '552_y', '553_y', '554_y', '555_y', '556_y', '557_y', '558_y', '559_y', '560_y', '561_y', '562_y', '563_y', '564_y', '565_y', '566_y', '567_y', '568_y', '569_y', '570_y', '571_y', '572_y', '573_y', '574_y', '575_y', '576_y', '577_y', '578_y', '579_y', '580_y', '581_y', '582_y', '583_y', '584_y', '585_y', '586_y', '587_y', '588_y', '589_y', '590_y', '591_y', '592_y', '593_y', '594_y', '595_y', '596_y', '597_y', '598_y', '599_y', '600_y', '601_y', '602_y', '603_y', '604_y', '605_y', '606_y', '607_y', '608_y', '609_y', '610_y', '611_y', '612_y', '613_y', '614_y', '615_y', '616_y', '617_y', '618_y', '619_y', '620_y', '621_y', '622_y', '623_y', '624_y', '625_y', '626_y', '627_y', '628_y', '629_y', '630_y', '631_y', '632_y', '633_y', '634_y', '635_y', '636_y', '637_y', '638_y', '639_y', '640_y', '641_y', '642_y', '643_y', '644_y', '645_y', '646_y', '647_y', '648_y', '649_y', '650_y', '651_y', '652_y', '653_y', '654_y', '655_y', '656_y', '657_y', '658_y', '659_y', '660_y', '661_y', '662_y', '663_y', '664_y', '665_y', '666_y', '667_y', '668_y', '669_y', '670_y', '671_y', '672_y', '673_y', '674_y', '675_y', '676_y', '677_y', '678_y', '679_y', '680_y', '681_y', '682_y', '683_y', '684_y', '685_y', '686_y', '687_y', '688_y', '689_y', '690_y', '691_y', '692_y', '693_y', '694_y', '695_y', '696_y', '697_y', '698_y', '699_y', '700_y', '701_y', '702_y', '703_y', '704_y', '705_y', '706_y', '707_y', '708_y', '709_y', '710_y', '711_y', '712_y', '713_y', '714_y', '715_y', '716_y', '717_y', '718_y', '719_y', '720_y', '721_y', '722_y', '723_y', '724_y', '725_y', '726_y', '727_y', '728_y', '729_y', '730_y', '731_y', '732_y', '733_y', '734_y', '735_y', '736_y', '737_y', '738_y', '739_y', '740_y', '741_y', '742_y', '743_y', '744_y', '745_y', '746_y', '747_y', '748_y', '749_y', '750_y', '751_y', '752_y', '753_y', '754_y', '755_y', '756_y', '757_y', '758_y', '759_y', '760_y', '761_y', '762_y', '763_y', '764_y', '765_y', '766_y', '767_y']

In [9]:
trainingData = trainingData[selectedFeatureList]
testingData = testingData[selectedFeatureList]

In [10]:
train, train_y = getFeaturesAndTargetRemoveDate(trainingData)
test, test_y = getFeaturesAndTargetRemoveDate(testingData)

getStartEndDateShape(trainingData)
getStartEndDateShape(testingData)

print('train',train.shape,'test',test.shape,'len(train_y)',len(train_y),'len(test_y)',len(test_y))

startDate 2015-05-04
endDate 2018-12-31
shape (1336, 822)
---------------------------
startDate 2019-01-01
endDate 2019-03-28
shape (87, 822)
---------------------------
train (1336, 820) test (87, 820) len(train_y) 1336 len(test_y) 87


In [11]:
df_News_train =trainingData[['news_len',
 'lm_positive_x',
 'lm_negative_x',
 'lm_polarity_x',
 'lm_subjectivity_x',
 'actvie_x',
 'passive_x',
 'weak_x',
 'strong_x',
 'anger_x',
 'joy_x',
 'suprise_x',
 'sadness_x',
 'disgust_x',
 'anticipation_x',
 'fear_x',
 'trust_x',
 'tb_polarity_x',
 'tb_subjectivity_x',
 'hiv4_positive_x',
 'hiv4_negative_x',
 'hiv4_polarity_x',
 'hiv4_subjectivity_x']]
df_NEWS_test =testingData[['news_len',
 'lm_positive_x',
 'lm_negative_x',
 'lm_polarity_x',
 'lm_subjectivity_x',
 'actvie_x',
 'passive_x',
 'weak_x',
 'strong_x',
 'anger_x',
 'joy_x',
 'suprise_x',
 'sadness_x',
 'disgust_x',
 'anticipation_x',
 'fear_x',
 'trust_x',
 'tb_polarity_x',
 'tb_subjectivity_x',
 'hiv4_positive_x',
 'hiv4_negative_x',
 'hiv4_polarity_x',
 'hiv4_subjectivity_x']]

In [12]:
df_FI_train = trainingData[['Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'sma2',
 'sma2_1',
 'sma2_increment',
 'sma2_1_increment',
 'vol_increment',
 'vol_rel_increment',
 'open_1',
 'open_incr',
 'ma7',
 'ma21',
 '26ema',
 '12ema',
 'MACD',
 '30 Day MA',
 '30 Day STD',
 'upper_band',
 'lower_band',
 'ema',
 'momentum',
 'log_momentum',
 'fft',
 'absolute',
 'angle']]
df_FI_test = testingData[['Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'sma2',
 'sma2_1',
 'sma2_increment',
 'sma2_1_increment',
 'vol_increment',
 'vol_rel_increment',
 'open_1',
 'open_incr',
 'ma7',
 'ma21',
 '26ema',
 '12ema',
 'MACD',
 '30 Day MA',
 '30 Day STD',
 'upper_band',
 'lower_band',
 'ema',
 'momentum',
 'log_momentum',
 'fft',
 'absolute',
 'angle']]

In [13]:
df_tweeter_train= trainingData[['0_y', '1_y', '2_y', '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y', '12_y', '13_y', '14_y', '15_y', '16_y', '17_y', '18_y', '19_y', '20_y', '21_y', '22_y', '23_y', '24_y', '25_y', '26_y', '27_y', '28_y', '29_y', '30_y', '31_y', '32_y', '33_y', '34_y', '35_y', '36_y', '37_y', '38_y', '39_y', '40_y', '41_y', '42_y', '43_y', '44_y', '45_y', '46_y', '47_y', '48_y', '49_y', '50_y', '51_y', '52_y', '53_y', '54_y', '55_y', '56_y', '57_y', '58_y', '59_y', '60_y', '61_y', '62_y', '63_y', '64_y', '65_y', '66_y', '67_y', '68_y', '69_y', '70_y', '71_y', '72_y', '73_y', '74_y', '75_y', '76_y', '77_y', '78_y', '79_y', '80_y', '81_y', '82_y', '83_y', '84_y', '85_y', '86_y', '87_y', '88_y', '89_y', '90_y', '91_y', '92_y', '93_y', '94_y', '95_y', '96_y', '97_y', '98_y', '99_y', '100_y', '101_y', '102_y', '103_y', '104_y', '105_y', '106_y', '107_y', '108_y', '109_y', '110_y', '111_y', '112_y', '113_y', '114_y', '115_y', '116_y', '117_y', '118_y', '119_y', '120_y', '121_y', '122_y', '123_y', '124_y', '125_y', '126_y', '127_y', '128_y', '129_y', '130_y', '131_y', '132_y', '133_y', '134_y', '135_y', '136_y', '137_y', '138_y', '139_y', '140_y', '141_y', '142_y', '143_y', '144_y', '145_y', '146_y', '147_y', '148_y', '149_y', '150_y', '151_y', '152_y', '153_y', '154_y', '155_y', '156_y', '157_y', '158_y', '159_y', '160_y', '161_y', '162_y', '163_y', '164_y', '165_y', '166_y', '167_y', '168_y', '169_y', '170_y', '171_y', '172_y', '173_y', '174_y', '175_y', '176_y', '177_y', '178_y', '179_y', '180_y', '181_y', '182_y', '183_y', '184_y', '185_y', '186_y', '187_y', '188_y', '189_y', '190_y', '191_y', '192_y', '193_y', '194_y', '195_y', '196_y', '197_y', '198_y', '199_y', '200_y', '201_y', '202_y', '203_y', '204_y', '205_y', '206_y', '207_y', '208_y', '209_y', '210_y', '211_y', '212_y', '213_y', '214_y', '215_y', '216_y', '217_y', '218_y', '219_y', '220_y', '221_y', '222_y', '223_y', '224_y', '225_y', '226_y', '227_y', '228_y', '229_y', '230_y', '231_y', '232_y', '233_y', '234_y', '235_y', '236_y', '237_y', '238_y', '239_y', '240_y', '241_y', '242_y', '243_y', '244_y', '245_y', '246_y', '247_y', '248_y', '249_y', '250_y', '251_y', '252_y', '253_y', '254_y', '255_y', '256_y', '257_y', '258_y', '259_y', '260_y', '261_y', '262_y', '263_y', '264_y', '265_y', '266_y', '267_y', '268_y', '269_y', '270_y', '271_y', '272_y', '273_y', '274_y', '275_y', '276_y', '277_y', '278_y', '279_y', '280_y', '281_y', '282_y', '283_y', '284_y', '285_y', '286_y', '287_y', '288_y', '289_y', '290_y', '291_y', '292_y', '293_y', '294_y', '295_y', '296_y', '297_y', '298_y', '299_y', '300_y', '301_y', '302_y', '303_y', '304_y', '305_y', '306_y', '307_y', '308_y', '309_y', '310_y', '311_y', '312_y', '313_y', '314_y', '315_y', '316_y', '317_y', '318_y', '319_y', '320_y', '321_y', '322_y', '323_y', '324_y', '325_y', '326_y', '327_y', '328_y', '329_y', '330_y', '331_y', '332_y', '333_y', '334_y', '335_y', '336_y', '337_y', '338_y', '339_y', '340_y', '341_y', '342_y', '343_y', '344_y', '345_y', '346_y', '347_y', '348_y', '349_y', '350_y', '351_y', '352_y', '353_y', '354_y', '355_y', '356_y', '357_y', '358_y', '359_y', '360_y', '361_y', '362_y', '363_y', '364_y', '365_y', '366_y', '367_y', '368_y', '369_y', '370_y', '371_y', '372_y', '373_y', '374_y', '375_y', '376_y', '377_y', '378_y', '379_y', '380_y', '381_y', '382_y', '383_y', '384_y', '385_y', '386_y', '387_y', '388_y', '389_y', '390_y', '391_y', '392_y', '393_y', '394_y', '395_y', '396_y', '397_y', '398_y', '399_y', '400_y', '401_y', '402_y', '403_y', '404_y', '405_y', '406_y', '407_y', '408_y', '409_y', '410_y', '411_y', '412_y', '413_y', '414_y', '415_y', '416_y', '417_y', '418_y', '419_y', '420_y', '421_y', '422_y', '423_y', '424_y', '425_y', '426_y', '427_y', '428_y', '429_y', '430_y', '431_y', '432_y', '433_y', '434_y', '435_y', '436_y', '437_y', '438_y', '439_y', '440_y', '441_y', '442_y', '443_y', '444_y', '445_y', '446_y', '447_y', '448_y', '449_y', '450_y', '451_y', '452_y', '453_y', '454_y', '455_y', '456_y', '457_y', '458_y', '459_y', '460_y', '461_y', '462_y', '463_y', '464_y', '465_y', '466_y', '467_y', '468_y', '469_y', '470_y', '471_y', '472_y', '473_y', '474_y', '475_y', '476_y', '477_y', '478_y', '479_y', '480_y', '481_y', '482_y', '483_y', '484_y', '485_y', '486_y', '487_y', '488_y', '489_y', '490_y', '491_y', '492_y', '493_y', '494_y', '495_y', '496_y', '497_y', '498_y', '499_y', '500_y', '501_y', '502_y', '503_y', '504_y', '505_y', '506_y', '507_y', '508_y', '509_y', '510_y', '511_y', '512_y', '513_y', '514_y', '515_y', '516_y', '517_y', '518_y', '519_y', '520_y', '521_y', '522_y', '523_y', '524_y', '525_y', '526_y', '527_y', '528_y', '529_y', '530_y', '531_y', '532_y', '533_y', '534_y', '535_y', '536_y', '537_y', '538_y', '539_y', '540_y', '541_y', '542_y', '543_y', '544_y', '545_y', '546_y', '547_y', '548_y', '549_y', '550_y', '551_y', '552_y', '553_y', '554_y', '555_y', '556_y', '557_y', '558_y', '559_y', '560_y', '561_y', '562_y', '563_y', '564_y', '565_y', '566_y', '567_y', '568_y', '569_y', '570_y', '571_y', '572_y', '573_y', '574_y', '575_y', '576_y', '577_y', '578_y', '579_y', '580_y', '581_y', '582_y', '583_y', '584_y', '585_y', '586_y', '587_y', '588_y', '589_y', '590_y', '591_y', '592_y', '593_y', '594_y', '595_y', '596_y', '597_y', '598_y', '599_y', '600_y', '601_y', '602_y', '603_y', '604_y', '605_y', '606_y', '607_y', '608_y', '609_y', '610_y', '611_y', '612_y', '613_y', '614_y', '615_y', '616_y', '617_y', '618_y', '619_y', '620_y', '621_y', '622_y', '623_y', '624_y', '625_y', '626_y', '627_y', '628_y', '629_y', '630_y', '631_y', '632_y', '633_y', '634_y', '635_y', '636_y', '637_y', '638_y', '639_y', '640_y', '641_y', '642_y', '643_y', '644_y', '645_y', '646_y', '647_y', '648_y', '649_y', '650_y', '651_y', '652_y', '653_y', '654_y', '655_y', '656_y', '657_y', '658_y', '659_y', '660_y', '661_y', '662_y', '663_y', '664_y', '665_y', '666_y', '667_y', '668_y', '669_y', '670_y', '671_y', '672_y', '673_y', '674_y', '675_y', '676_y', '677_y', '678_y', '679_y', '680_y', '681_y', '682_y', '683_y', '684_y', '685_y', '686_y', '687_y', '688_y', '689_y', '690_y', '691_y', '692_y', '693_y', '694_y', '695_y', '696_y', '697_y', '698_y', '699_y', '700_y', '701_y', '702_y', '703_y', '704_y', '705_y', '706_y', '707_y', '708_y', '709_y', '710_y', '711_y', '712_y', '713_y', '714_y', '715_y', '716_y', '717_y', '718_y', '719_y', '720_y', '721_y', '722_y', '723_y', '724_y', '725_y', '726_y', '727_y', '728_y', '729_y', '730_y', '731_y', '732_y', '733_y', '734_y', '735_y', '736_y', '737_y', '738_y', '739_y', '740_y', '741_y', '742_y', '743_y', '744_y', '745_y', '746_y', '747_y', '748_y', '749_y', '750_y', '751_y', '752_y', '753_y', '754_y', '755_y', '756_y', '757_y', '758_y', '759_y', '760_y', '761_y', '762_y', '763_y', '764_y', '765_y', '766_y', '767_y']
    
]
df_tweetwer_test =testingData[['0_y', '1_y', '2_y', '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y', '12_y', '13_y', '14_y', '15_y', '16_y', '17_y', '18_y', '19_y', '20_y', '21_y', '22_y', '23_y', '24_y', '25_y', '26_y', '27_y', '28_y', '29_y', '30_y', '31_y', '32_y', '33_y', '34_y', '35_y', '36_y', '37_y', '38_y', '39_y', '40_y', '41_y', '42_y', '43_y', '44_y', '45_y', '46_y', '47_y', '48_y', '49_y', '50_y', '51_y', '52_y', '53_y', '54_y', '55_y', '56_y', '57_y', '58_y', '59_y', '60_y', '61_y', '62_y', '63_y', '64_y', '65_y', '66_y', '67_y', '68_y', '69_y', '70_y', '71_y', '72_y', '73_y', '74_y', '75_y', '76_y', '77_y', '78_y', '79_y', '80_y', '81_y', '82_y', '83_y', '84_y', '85_y', '86_y', '87_y', '88_y', '89_y', '90_y', '91_y', '92_y', '93_y', '94_y', '95_y', '96_y', '97_y', '98_y', '99_y', '100_y', '101_y', '102_y', '103_y', '104_y', '105_y', '106_y', '107_y', '108_y', '109_y', '110_y', '111_y', '112_y', '113_y', '114_y', '115_y', '116_y', '117_y', '118_y', '119_y', '120_y', '121_y', '122_y', '123_y', '124_y', '125_y', '126_y', '127_y', '128_y', '129_y', '130_y', '131_y', '132_y', '133_y', '134_y', '135_y', '136_y', '137_y', '138_y', '139_y', '140_y', '141_y', '142_y', '143_y', '144_y', '145_y', '146_y', '147_y', '148_y', '149_y', '150_y', '151_y', '152_y', '153_y', '154_y', '155_y', '156_y', '157_y', '158_y', '159_y', '160_y', '161_y', '162_y', '163_y', '164_y', '165_y', '166_y', '167_y', '168_y', '169_y', '170_y', '171_y', '172_y', '173_y', '174_y', '175_y', '176_y', '177_y', '178_y', '179_y', '180_y', '181_y', '182_y', '183_y', '184_y', '185_y', '186_y', '187_y', '188_y', '189_y', '190_y', '191_y', '192_y', '193_y', '194_y', '195_y', '196_y', '197_y', '198_y', '199_y', '200_y', '201_y', '202_y', '203_y', '204_y', '205_y', '206_y', '207_y', '208_y', '209_y', '210_y', '211_y', '212_y', '213_y', '214_y', '215_y', '216_y', '217_y', '218_y', '219_y', '220_y', '221_y', '222_y', '223_y', '224_y', '225_y', '226_y', '227_y', '228_y', '229_y', '230_y', '231_y', '232_y', '233_y', '234_y', '235_y', '236_y', '237_y', '238_y', '239_y', '240_y', '241_y', '242_y', '243_y', '244_y', '245_y', '246_y', '247_y', '248_y', '249_y', '250_y', '251_y', '252_y', '253_y', '254_y', '255_y', '256_y', '257_y', '258_y', '259_y', '260_y', '261_y', '262_y', '263_y', '264_y', '265_y', '266_y', '267_y', '268_y', '269_y', '270_y', '271_y', '272_y', '273_y', '274_y', '275_y', '276_y', '277_y', '278_y', '279_y', '280_y', '281_y', '282_y', '283_y', '284_y', '285_y', '286_y', '287_y', '288_y', '289_y', '290_y', '291_y', '292_y', '293_y', '294_y', '295_y', '296_y', '297_y', '298_y', '299_y', '300_y', '301_y', '302_y', '303_y', '304_y', '305_y', '306_y', '307_y', '308_y', '309_y', '310_y', '311_y', '312_y', '313_y', '314_y', '315_y', '316_y', '317_y', '318_y', '319_y', '320_y', '321_y', '322_y', '323_y', '324_y', '325_y', '326_y', '327_y', '328_y', '329_y', '330_y', '331_y', '332_y', '333_y', '334_y', '335_y', '336_y', '337_y', '338_y', '339_y', '340_y', '341_y', '342_y', '343_y', '344_y', '345_y', '346_y', '347_y', '348_y', '349_y', '350_y', '351_y', '352_y', '353_y', '354_y', '355_y', '356_y', '357_y', '358_y', '359_y', '360_y', '361_y', '362_y', '363_y', '364_y', '365_y', '366_y', '367_y', '368_y', '369_y', '370_y', '371_y', '372_y', '373_y', '374_y', '375_y', '376_y', '377_y', '378_y', '379_y', '380_y', '381_y', '382_y', '383_y', '384_y', '385_y', '386_y', '387_y', '388_y', '389_y', '390_y', '391_y', '392_y', '393_y', '394_y', '395_y', '396_y', '397_y', '398_y', '399_y', '400_y', '401_y', '402_y', '403_y', '404_y', '405_y', '406_y', '407_y', '408_y', '409_y', '410_y', '411_y', '412_y', '413_y', '414_y', '415_y', '416_y', '417_y', '418_y', '419_y', '420_y', '421_y', '422_y', '423_y', '424_y', '425_y', '426_y', '427_y', '428_y', '429_y', '430_y', '431_y', '432_y', '433_y', '434_y', '435_y', '436_y', '437_y', '438_y', '439_y', '440_y', '441_y', '442_y', '443_y', '444_y', '445_y', '446_y', '447_y', '448_y', '449_y', '450_y', '451_y', '452_y', '453_y', '454_y', '455_y', '456_y', '457_y', '458_y', '459_y', '460_y', '461_y', '462_y', '463_y', '464_y', '465_y', '466_y', '467_y', '468_y', '469_y', '470_y', '471_y', '472_y', '473_y', '474_y', '475_y', '476_y', '477_y', '478_y', '479_y', '480_y', '481_y', '482_y', '483_y', '484_y', '485_y', '486_y', '487_y', '488_y', '489_y', '490_y', '491_y', '492_y', '493_y', '494_y', '495_y', '496_y', '497_y', '498_y', '499_y', '500_y', '501_y', '502_y', '503_y', '504_y', '505_y', '506_y', '507_y', '508_y', '509_y', '510_y', '511_y', '512_y', '513_y', '514_y', '515_y', '516_y', '517_y', '518_y', '519_y', '520_y', '521_y', '522_y', '523_y', '524_y', '525_y', '526_y', '527_y', '528_y', '529_y', '530_y', '531_y', '532_y', '533_y', '534_y', '535_y', '536_y', '537_y', '538_y', '539_y', '540_y', '541_y', '542_y', '543_y', '544_y', '545_y', '546_y', '547_y', '548_y', '549_y', '550_y', '551_y', '552_y', '553_y', '554_y', '555_y', '556_y', '557_y', '558_y', '559_y', '560_y', '561_y', '562_y', '563_y', '564_y', '565_y', '566_y', '567_y', '568_y', '569_y', '570_y', '571_y', '572_y', '573_y', '574_y', '575_y', '576_y', '577_y', '578_y', '579_y', '580_y', '581_y', '582_y', '583_y', '584_y', '585_y', '586_y', '587_y', '588_y', '589_y', '590_y', '591_y', '592_y', '593_y', '594_y', '595_y', '596_y', '597_y', '598_y', '599_y', '600_y', '601_y', '602_y', '603_y', '604_y', '605_y', '606_y', '607_y', '608_y', '609_y', '610_y', '611_y', '612_y', '613_y', '614_y', '615_y', '616_y', '617_y', '618_y', '619_y', '620_y', '621_y', '622_y', '623_y', '624_y', '625_y', '626_y', '627_y', '628_y', '629_y', '630_y', '631_y', '632_y', '633_y', '634_y', '635_y', '636_y', '637_y', '638_y', '639_y', '640_y', '641_y', '642_y', '643_y', '644_y', '645_y', '646_y', '647_y', '648_y', '649_y', '650_y', '651_y', '652_y', '653_y', '654_y', '655_y', '656_y', '657_y', '658_y', '659_y', '660_y', '661_y', '662_y', '663_y', '664_y', '665_y', '666_y', '667_y', '668_y', '669_y', '670_y', '671_y', '672_y', '673_y', '674_y', '675_y', '676_y', '677_y', '678_y', '679_y', '680_y', '681_y', '682_y', '683_y', '684_y', '685_y', '686_y', '687_y', '688_y', '689_y', '690_y', '691_y', '692_y', '693_y', '694_y', '695_y', '696_y', '697_y', '698_y', '699_y', '700_y', '701_y', '702_y', '703_y', '704_y', '705_y', '706_y', '707_y', '708_y', '709_y', '710_y', '711_y', '712_y', '713_y', '714_y', '715_y', '716_y', '717_y', '718_y', '719_y', '720_y', '721_y', '722_y', '723_y', '724_y', '725_y', '726_y', '727_y', '728_y', '729_y', '730_y', '731_y', '732_y', '733_y', '734_y', '735_y', '736_y', '737_y', '738_y', '739_y', '740_y', '741_y', '742_y', '743_y', '744_y', '745_y', '746_y', '747_y', '748_y', '749_y', '750_y', '751_y', '752_y', '753_y', '754_y', '755_y', '756_y', '757_y', '758_y', '759_y', '760_y', '761_y', '762_y', '763_y', '764_y', '765_y', '766_y', '767_y']]

In [14]:
# Naive Bayes for Baseline
from sklearn.naive_bayes import GaussianNB,MultinomialNB
def NaiveBayes(train_features , train_labels, test_features,test_labels):
    gnb = GaussianNB()
    y_pred = gnb.fit(train_features, train_labels).predict(test_features)
    print('accuracy_score',accuracy_score(y_pred, test_labels))
    print('classification_score\n',classification_report(y_pred, test_labels))
    return gnb.predict(train_features),gnb.predict(test_features)

In [15]:
def RF(train_features , train_labels, test_features,test_labels, dispImpOfFeature = False):

        rf_model =RandomForestClassifier(n_estimators=5000, max_depth=4,random_state=0)
        rf_model.fit(train_features, train_labels)
        y_predicted=rf_model.predict(test_features)
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++' )
        print('accuracy_score',accuracy_score(y_predicted, test_labels))
        print('classification_score\n',classification_report(y_predicted, test_labels))
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        return rf_model.predict(train_features),rf_model.predict(test_features)

In [16]:
def DenseNetwork(featureLength):
    #defifne a sequentail Model
    model = Sequential()

    #Hidden Layer-1
    model.add(Dense(100,activation='relu',input_dim=featureLength,kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3, noise_shape=None, seed=None))

    #Hidden Layer-2
    model.add(Dense(100,activation = 'relu',kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3, noise_shape=None, seed=None))

    #Output layer
    model.add(Dense(1,activation='sigmoid'))
    model.summary()
    return model

In [17]:
def evaluateModelAndTestAccuracy(model,trainX, trainY, testX, testY,n_epoch):
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(trainX, trainY, epochs=n_epoch, validation_split=0.1,batch_size=256, verbose=1,)
    
    results = model.evaluate(testX, testY)
    
    y_pred = model.predict_classes(testX)
    print("Evaluation on test data: Loss - {0},  Acurracy - {1}".format(results[0],str(results[1]*100)))
    print('classification_score\n',classification_report(y_pred, testY))
    return history

In [18]:
def vizModelMetric(history,n_epoch):
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, n_epoch), history.history["loss"], label="train_loss")
    plt.plot(np.arange(0, n_epoch), history.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, n_epoch), history.history["acc"], label="train_acc")
    plt.plot(np.arange(0, n_epoch), history.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()

In [19]:
# Building Train and test dataset over News

In [21]:
train_fnw, test_fnw = NaiveBayes(df_News_train , train_y, df_NEWS_test,test_y)

accuracy_score 0.41379310344827586
classification_score
               precision    recall  f1-score   support

         0.0       0.75      0.39      0.51        69
         1.0       0.18      0.50      0.26        18

   micro avg       0.41      0.41      0.41        87
   macro avg       0.46      0.45      0.39        87
weighted avg       0.63      0.41      0.46        87



In [ ]:
#Building Train and test dataset over Twitter

In [ ]:
len(test_y)

In [ ]:
train_ftw, test_ftw =RF(df_FinancialIndicator_train , train_y, df_FinancialIndicator_test,test_y)

In [ ]:
#Building Train and test dataset overt Financial Indicator

In [ ]:
featureLength = .shape[1]
vizModelMetric(evaluateModelAndTestAccuracy(DenseNetwork(featureLength),train, train_y, test, test_y ,n_epoch),n_epoch)